In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("Data/Sleep_health_and_lifestyle_dataset.csv")

df

,Person ID,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Blood Pressure,Heart Rate,Daily Steps,Sleep Disorder
0,1,Male,27,Software Engineer,6.1,6,42,6,Overweight,126/83,77,4200,NaN
1,2,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,NaN
2,3,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,NaN
3,4,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea
4,5,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea
...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,370,Female,59,Nurse,8.1,9,75,3,Overweight,140/95,68,7000,Sleep Apnea
370,371,Female,59,Nurse,8.0,9,75,3,Overweight,140/95,68,7000,Sleep Apnea
371,372,Female,59,Nurse,8.1,9,75,3,Overweight,140/95,68,7000,Sleep Apnea
372,373,Female,59,Nurse,8.1,9,75,3,Overweight,140/95,68,7000,Sleep Apnea


In [ ]:
import sqlite3

In [ ]:
open("Data/health.sqlite", "x")

In [ ]:
con = sqlite3.connect("Data/health.db")
cur = con.cursor()

In [ ]:
cur.execute("CREATE TABLE person(id, age, gender_id, occupation_id)")

In [3]:
person = df[["Person ID", "Gender", "Age", "Occupation"]]

person

,Person ID,Gender,Age,Occupation
0,1,Male,27,Software Engineer
1,2,Male,28,Doctor
2,3,Male,28,Doctor
3,4,Male,28,Sales Representative
4,5,Male,28,Sales Representative
...,...,...,...,...
369,370,Female,59,Nurse
370,371,Female,59,Nurse
371,372,Female,59,Nurse
372,373,Female,59,Nurse


In [6]:
gender_list = list(df["Gender"].unique())

gender_dict = {"id": [i for i in range(len(gender_list))], "gender_name": [i for i in gender_list]}

gender = pd.DataFrame(gender_dict)

gender.to_csv("Data/gender.csv")

In [5]:
occupation_list = list(df["Occupation"].unique())

occupation_dict = {"id": [i for i in range(len(occupation_list))], "occupation": [i for i in occupation_list]}

occupation = pd.DataFrame(occupation_dict)

occupation.to_csv("Data/occupation.csv")


In [10]:
person_gender_occupation = person.merge(gender, left_on = "Gender",right_on = "gender_name", how = "inner").merge(occupation, left_on = "Occupation", right_on = "occupation", how = "inner")

person_cleaned = person_gender_occupation[["Person ID", "id_x", "Age", "id_y"]].rename(columns = {"Person ID": "id", "id_x": "gender_id", "Age": "age", "id_y": "occupation_id"})

person_cleaned.to_csv("Data/person.csv")

In [11]:
df["Sleep Disorder"].fillna(value = "None", axis = 0, inplace = True)

sleep_disorder_list = list(df["Sleep Disorder"].unique())

sleep_disorder_dict = {"id": [i for i in range(len(sleep_disorder_list))], "sleep_disorder": [i for i in sleep_disorder_list]}

sleep_disorder = pd.DataFrame(sleep_disorder_dict)

sleep_disorder.to_csv("Data/sleep_disorder.csv")

In [12]:
sleep_df = df[["Person ID", "Sleep Duration", "Quality of Sleep", "Sleep Disorder"]]

sleep_cleaned = sleep_df.merge(sleep_disorder, left_on = "Sleep Disorder", right_on = "sleep_disorder", how = "inner")

sleep = sleep_cleaned[["Person ID", "Sleep Duration", "Quality of Sleep", "id"]].reset_index()\
    .rename(columns = {"index": "id", "Person ID": "person_id", "Sleep Duration": "sleep_duration", "id": "sleep_disorder_id"})

sleep.to_csv("Data/sleep.csv")

In [13]:
activity_df = df[["Person ID", "Physical Activity Level", "Daily Steps"]].reset_index()\
    .rename(columns = {"index": "id", "Person ID": "person_id", "Physical Activity Level": "activity_level", "Daily Steps": "daily_steps"})

activity_df.to_csv("Data/activity.csv")

In [19]:
df["BMI Category"] = np.where(df["BMI Category"] == "Normal Weight", "Normal", df["BMI Category"])

In [23]:
bmi_list = list(df["BMI Category"].unique())

bmi_dict = {"id": [i for i in range(len(bmi_list))], "bmi_name": [i for i in bmi_list]}

bmi_df = pd.DataFrame(bmi_dict)

bmi_df.to_csv("Data/bmi.csv")

In [24]:
df[["Systolic", "Diastolic"]] = df["Blood Pressure"].str.split("/", expand = True)

In [51]:
df[["Systolic", "Diastolic"]] = df[["Systolic", "Diastolic"]].astype({"Systolic": "int64", "Diastolic": "int64"})

In [61]:
health_df = df[["Person ID", "BMI Category", "Blood Pressure", "Stress Level", "Heart Rate", "Systolic", "Diastolic"]]

health_cleaned = health_df.merge(bmi_df, left_on = "BMI Category", right_on = "bmi_name", how = "inner")

health = health_cleaned[["Person ID", "Blood Pressure", "Stress Level", "Heart Rate", "Systolic", "Diastolic", "id"]].reset_index()\
    .rename(columns = {"index": "id", 
                       "Person ID": "person_id",
                       "Blood Pressure": "blood_pressure",
                       "Stress Level": "stress_level",
                       "Heart Rate": "heart_rate",
                       "Systolic": "systolic",
                       "Diastolic": "diastolic",
                       "id": "bmi_id"
                       })
    
health.to_csv("Data/health.csv")

In [58]:
bmi_df

,id,bmi_name
0,0,Overweight
1,1,Normal
2,2,Obese


In [53]:
df

,Person ID,Gender,Age,Occupation,Sleep Duration,Quality of Sleep,Physical Activity Level,Stress Level,BMI Category,Blood Pressure,Heart Rate,Daily Steps,Sleep Disorder,Systolic,Diastolic
0,1,Male,27,Software Engineer,6.1,6,42,6,Overweight,126/83,77,4200,None,126,83
1,2,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,None,125,80
2,3,Male,28,Doctor,6.2,6,60,8,Normal,125/80,75,10000,None,125,80
3,4,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea,140,90
4,5,Male,28,Sales Representative,5.9,4,30,8,Obese,140/90,85,3000,Sleep Apnea,140,90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,370,Female,59,Nurse,8.1,9,75,3,Overweight,140/95,68,7000,Sleep Apnea,140,95
370,371,Female,59,Nurse,8.0,9,75,3,Overweight,140/95,68,7000,Sleep Apnea,140,95
371,372,Female,59,Nurse,8.1,9,75,3,Overweight,140/95,68,7000,Sleep Apnea,140,95
372,373,Female,59,Nurse,8.1,9,75,3,Overweight,140/95,68,7000,Sleep Apnea,140,95
